In [ ]:
###############################################################
# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)
###############################################################

# 1. İş Problemi (Business Problem)
# 2. Veriyi Anlama (Data Understanding)
# 3. Veri Hazırlama (Data Preparation)
# 4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
# 5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
# 6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
# 7. Tüm Sürecin Fonksiyonlaştırılması

###############################################################
# 1. İş Problemi (Business Problem)
###############################################################

# Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre
# pazarlama stratejileri belirlemek istiyor.

# Veri Seti Hikayesi
# https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

# Değişkenler
#
# InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
# StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
# Description: Ürün ismi
# Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
# InvoiceDate: Fatura tarihi ve zamanı.
# UnitPrice: Ürün fiyatı (Sterlin cinsinden)
# CustomerID: Eşsiz müşteri numarası
# Country: Ülke ismi. Müşterinin yaşadığı ülke.


In [1]:

###############################################################
# 2. Veriyi Anlama (Data Understanding)
###############################################################

import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_ = pd.read_excel("/kaggle/input/hackathon-odeal/hackathon_data.xlsx")
df = df_.copy()
df.head(10)
df.shape
df.isnull().sum()

# # essiz urun sayisi nedir?
# df["Description"].nunique()

# df["Description"].value_counts().head()

# df.groupby("Description").agg({"Quantity": "sum"}).head(10)

# df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

# df["Invoice"].nunique()

# df["TotalPrice"] = df["Quantity"] * df["Price"]

# df.groupby("Invoice").agg({"TotalPrice": "sum"}).head()



id                         0
Marka                      0
IsyeriTipi                 0
UyeDurum                   0
UyeAdres                   0
Sehir                      0
UyeAktivasyonTarih         0
IslemID                    0
IslemTarih                 0
IslemTutar                 0
Tercih                     0
Versiyon              976357
SurumTarih            976357
dtype: int64

In [3]:
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 400)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
df.head(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1000000 non-null  int64         
 1   Marka               1000000 non-null  object        
 2   IsyeriTipi          1000000 non-null  object        
 3   UyeDurum            1000000 non-null  object        
 4   UyeAdres            1000000 non-null  object        
 5   Sehir               1000000 non-null  object        
 6   UyeAktivasyonTarih  1000000 non-null  datetime64[ns]
 7   IslemID             1000000 non-null  int64         
 8   IslemTarih          1000000 non-null  datetime64[ns]
 9   IslemTutar          1000000 non-null  object        
 10  Tercih              1000000 non-null  object        
 11  Versiyon            23643 non-null    object        
 12  SurumTarih          23643 non-null    datetime64[ns]
dtypes: datetime64

In [7]:


def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
#     print("##################### Quantiles #####################")
#     print(dataframe.quantile([0, 0.05, 0.50, 0.75, 0.95, 0.99, 1]).T)

check_df(df)

def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car


cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=4, car_th=20)


check_df(df)

##################### Shape #####################
(1000000, 13)
##################### Types #####################
id                             int64
Marka                         object
IsyeriTipi                    object
UyeDurum                      object
UyeAdres                      object
Sehir                         object
UyeAktivasyonTarih    datetime64[ns]
IslemID                        int64
IslemTarih            datetime64[ns]
IslemTutar                    object
Tercih                        object
Versiyon                      object
SurumTarih            datetime64[ns]
dtype: object
##################### Head #####################
          id          Marka     IsyeriTipi UyeDurum                                  UyeAdres   Sehir  UyeAktivasyonTarih    IslemID          IslemTarih IslemTutar    Tercih Versiyon SurumTarih
0  301002470  Kzar kü pazar  Tüzel Şirket    Aktif  KALE MAH. GAZİ CAD. 82 C İLKADIM/ SAMSUN  Samsun 2015-09-08 15:08:00  401003136 2015-09-18 14:3

In [ ]:
################################
# Adım 2: Müşterileri segmentlerken kullanacağınız değişkenleri seçiniz.
# Not: Tenure (Müşterinin yaşı), Recency (en son kaç gün önce alışveriş yaptığı) gibi yeni değişkenler oluşturabilirsiniz.
################################

df['tenure'] = (pd.to_datetime('today') - pd.to_datetime(df['first_order_date'])).dt.days
# Müşterinin e-ticaret sitesi/mağaza ile ilk kontağından bu yana geçen zaman olarak geçer.
# Bugünden minimum kontak tarihi çıkartılmasıyla bulunabilir.

df['recency'] = (pd.to_datetime('today') - pd.to_datetime(df['last_order_date'])).dt.days
# Müşterinin ne kadar zamandır websitesinden/mağazadan hizmet aldığı, ne zamandır üye olduğu gibi bilgileri verir.
# Hesaplanması genellikle, bugünden son üyelik tarihi/son sipariş tarihinin çıkartılmasıyla elde edilir.

df['frequency'] = df['order_num_total_ever_online'] + df['order_num_total_ever_offline']
# Müşterinin ne sıklıkla alışveriş yaptığını, ne sıklıkla siteye giriş yaptığını gösteren metriktir.
# Genellikle sipariş numarası/sipariş kodunun saydırılmasıyla sonuç verir.

df['monetary'] = df['customer_value_total_ever_online'] + df['customer_value_total_ever_offline']
# Müşterinin harcamalarının toplamıdır. E-ticaret sitesine getirdiği ciro,
# aldığı hizmetler sonrası toplanan getiri olarak da tanımlanır.

df.info()
df.head()
df.shape



In [ ]:
################################
# Görev 2: K-Means ile Müşteri Segmentasyonu
################################

def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

cat_cols, num_cols, cat_but_car = grab_col_names(df, cat_th=4, car_th=20)

df[num_cols].describe().T


def cat_summary(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show(block=True)


for col in cat_cols:
    cat_summary(df, col, plot=False)


def one_hot_encoder(df, categorical_cols, drop_first=False):
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=drop_first)
    return df

df = one_hot_encoder(df, cat_cols, drop_first=True)

df.shape
df.head()


In [ ]:
################################
# Adım 1: Değişkenleri standartlaştırınız.
################################

sc = MinMaxScaler((0, 1))
df_new = sc.fit_transform(df[num_cols])
df_new = pd.DataFrame(df_new, columns=num_cols)
df_new.info()
df_new.shape
df_new = df_new.merge(df.iloc[:, -7:], left_index=True, right_index=True)

df_new.head()




In [ ]:
################################
# Adım 2: Optimum küme sayısını belirleyiniz.Optimum Küme Sayısının Belirlenmesi
################################

kmeans = KMeans()
ssd = []
K = range(1, 40)

for k in K:
    kmeans = KMeans(n_clusters=k).fit(df_new)
    ssd.append(kmeans.inertia_)

kmeans.get_params()
kmeans.cluster_centers_
len(kmeans.labels_)
kmeans.inertia_


plt.plot(K, ssd, "bx-")
plt.xlabel("Farklı K Değerlerine Karşılık SSE/SSR/SSD")
plt.title("Optimum Küme sayısı için Elbow Yöntemi")
plt.show()

kmeans = KMeans()
elbow = KElbowVisualizer(kmeans, k=(2, 40))
elbow.fit(df_new)
elbow.show(block=True)

elbow.elbow_value_


In [ ]:
################################
# Adım 3: Modelinizi oluşturunuz ve müşterilerinizi segmentleyiniz¶
################################

kmeans = KMeans(n_clusters=elbow.elbow_value_).fit(df_new)

kmeans.n_clusters
kmeans.cluster_centers_
kmeans.labels_

# Etiketlerin değişken atamasını yapıyoruz.
clusters_kmeans = kmeans.labels_

# Mevcut DF içerisine küme sayılarını ekliyoruz.
df["cluster"] = clusters_kmeans

df["cluster"] = df["cluster"] + 1
df.head()


# Kümelere göre dağılımını kontrol edelim
def Ratio_(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show(block=True)

Ratio_(df, "cluster")


In [ ]:
################################
# Adım 4: Herbir segmenti istatistiksel olarak inceleyeniz.
################################

df.groupby("cluster").agg(["mean", "median", "count", "sum"]).T

################################
# GÖREV 3: HİERARCHİCAL CLUSTERİNG İLE MÜŞTERİ SEGMENTASYONU
################################


In [ ]:
################################
# Adım 1: Görev 2'de standartlaştırdığınız dataframe'i kullanarak optimum küme sayısını belirleyiniz.
################################


hc_ward = linkage(df_new, "ward")

plt.figure(figsize=(10, 5))
plt.title("Hiyerarşik Kümeleme Dendogramı")
plt.xlabel("Gözlem Birimleri")
plt.ylabel("Uzaklıklar")
dendrogram(hc_ward,
            truncate_mode="lastp",
            p=7,
            show_contracted=True,
            leaf_font_size=10)
plt.show(block=True)

plt.figure(figsize=(10, 5))
plt.title("Hiyerarşik Kümeleme Dendogramı")
dend = dendrogram(hc_ward,
                 truncate_mode="lastp",
                  p=7,
                  show_contracted=True,
                  leaf_font_size=10)
plt.axhline(y=60, color="b", linestyle="--")
plt.axhline(y=50, color="r", linestyle="--")
plt.show(block=True)


In [ ]:
################################
# Adım 2: Modelinizi oluşturunuz ve müşterileriniz segmentleyiniz.
################################

from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=6, linkage="average")

clusters = cluster.fit_predict(df_new)

df["hi_cluster_no"] = clusters

df["hi_cluster_no"] = df["hi_cluster_no"] + 1

df.head()

# Kümelere göre dağılımını kontrol edelim
def Ratio_(dataframe, col_name, plot=False):
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}))
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show(block=True)


Ratio_(df, "hi_cluster_no")

In [ ]:
################################
# Adım 3: Her bir segmenti istatistiksel olarak inceleyeniz.
################################

df.groupby("hi_cluster_no").agg(["mean", "median", "count", "sum"]).T